The idea with a regression model is to estimate the price and assign a category based on thresholds that define the bins.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

# Load data

In [2]:
data = pd.read_csv("../../data/raw/listings.csv", usecols=['id','accommodates','room_type','beds','bedrooms','bathrooms_text','neighbourhood_group_cleansed','amenities','latitude','longitude','price'])
print(data.shape)
data.head(3)

(38277, 11)


,id,neighbourhood_group_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price
0,2595,Manhattan,40.75356,-73.98559,Entire home/apt,1,1 bath,NaN,1.0,"[""Extra pillows and blankets"", ""Baking sheet"",...",$150.00
1,3831,Brooklyn,40.68494,-73.95765,Entire home/apt,3,1 bath,1.0,3.0,"[""Extra pillows and blankets"", ""Luggage dropof...",$75.00
2,5121,Brooklyn,40.68535,-73.95512,Private room,2,NaN,1.0,1.0,"[""Kitchen"", ""Long term stays allowed"", ""Wifi"",...",$60.00


# Feature engineer, cleaning and transformation

In [3]:
data['price_processed'] = data.price.str[1:-3].str.replace(',','').astype(int)
data.drop(columns=['price'], inplace=True)
data.rename(columns={'price_processed':'price'}, inplace=True)

In [4]:
def extract_num_baths(text):
    found_groups = [group for group in re.findall("(\d*\.?\d*).*", str(text)) if group!='']
    if found_groups:
        return float(found_groups[0])
    elif pd.isna(text) or text is None:
        return np.nan
    # Special cases
    elif "half-bath" in text.lower():
        return 0.5
    else:  # In case there were any cases like this
        return 1
    
data['bathrooms'] = data.bathrooms_text.apply(extract_num_baths)
data.drop(columns=['bathrooms_text'], inplace=True)

In [5]:
data.rename(columns={'neighbourhood_group_cleansed':'neighbourhood'}, inplace=True)

In [6]:
def find_amenities(lst):
    return pd.Series([int('tv' in lst.lower()), int('elevator' in lst.lower()), int('wifi' in lst.lower())])

data[['tv','elevator','internet']] = data.amenities.apply(find_amenities)
data.drop(columns=['amenities'], inplace=True)

## Impute missing values

In [7]:
data.isna().sum()

id                  0
neighbourhood       0
latitude            0
longitude           0
room_type           0
accommodates        0
bedrooms         3975
beds             2405
price               0
bathrooms         107
tv                  0
elevator            0
internet            0
dtype: int64

In [8]:
median_num_baths_by_bedrooms = data.groupby('bedrooms')['bathrooms'].median()
median_num_baths_by_accommodates = data.groupby('accommodates')['bathrooms'].median().fillna(0)

data.loc[data.bathrooms.isna() & data.bedrooms.notna(), 'bathrooms'] = data.loc[data.bathrooms.isna() & data.bedrooms.notna(), 'bedrooms'].apply(lambda n_bedrooms : median_num_baths_by_bedrooms[n_bedrooms])
data.loc[data.bathrooms.isna(), 'bathrooms'] = data.loc[data.bathrooms.isna(), 'accommodates'].apply(lambda n_accommodates : median_num_baths_by_accommodates[n_accommodates])

data.bathrooms.isna().sum()

0

## Preprocess categorical variables

In [9]:
data.loc[ data.room_type.isin(['Private room','Shared room']), 'room_type' ] = "Private-Share room"

In [10]:
data.loc[ data.neighbourhood.isin(['Bronx','Queens','Staten Island']), 'neighbourhood' ] = "Bronx-Queens-StatenIsland"

In [11]:
data = pd.get_dummies(data, columns=['room_type','neighbourhood'], drop_first=True)

In [12]:
data.drop(columns=['id','beds','bedrooms'], inplace=True)

In [13]:
data.head()

,latitude,longitude,accommodates,price,bathrooms,tv,elevator,internet,room_type_Hotel room,room_type_Private-Share room,neighbourhood_Brooklyn,neighbourhood_Manhattan
0,40.75356,-73.98559,1,150,1.0,1,0,1,0,0,0,1
1,40.68494,-73.95765,3,75,1.0,1,0,1,0,0,1,0
2,40.68535,-73.95512,2,60,1.0,0,0,1,0,1,1,0
3,40.66265,-73.99454,4,275,1.5,1,0,1,0,0,1,0
4,40.76457,-73.98317,2,68,1.0,1,0,1,0,1,0,1


# Modeling

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
X = data.drop(columns=['price'])
y = data['price']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((30621, 11), (7656, 11), (30621,), (7656,))

## Baseline model

### Training

In [17]:
%%time
baseline_model = RandomForestRegressor(n_estimators=100,
                                       max_depth=4,
                                       random_state=0)

baseline_model = baseline_model.fit(X_train, y_train)

CPU times: user 1.99 s, sys: 15.1 ms, total: 2 s
Wall time: 2.05 s


### Evaluation

In [18]:
y_pred = baseline_model.predict(X_test)

In [19]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error: ", mae)
print("Mean Squared Error: ", mse)
print("Root Mean Squared Error: ", np.sqrt(mse))
print("R-squared: ", r2)

Mean Absolute Error:  86.80190506870406
Mean Squared Error:  112035.92707360614
Root Mean Squared Error:  334.71768264256093
R-squared:  0.11677114734975702


R-squared:
- A value of 0 means that the model explains none of the variance in the dependent variable.
- A value of 0.5 means that the model explains about 50% of the variance in the dependent variable.
- A value of 1 means that the model explains all of the variance in the dependent variable.

## Cross-validation & Grid search - RandomForestRegressor

In [22]:
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from tqdm import tqdm

In [74]:
params_GridSearch = {'n_estimators': [100,200,500],
                     'max_depth': [2,4,8,14,20],
                     'min_samples_leaf': [1,10,50],
                     'max_features': [1,2]}

In [75]:
def random_forest_cross_val(X, y, params, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    rmse = []
    
    for tr_ind, val_ind in skf.split(X, y):
        X_tr = X.iloc[tr_ind]
        y_tr = y.iloc[tr_ind]
        
        X_val = X.iloc[val_ind]
        y_val = y.iloc[val_ind]
        
        # print(X_tr.shape, y_tr.shape, X_val.shape, y_val.shape)
        
        model = RandomForestRegressor(n_estimators=params['n_estimators'],
                                      max_depth=params['max_depth'],
                                      min_samples_leaf=params['min_samples_leaf'],
                                      max_features=params['max_features'],
                                      n_jobs=-1, random_state=42)
        
        model.fit(X_tr, y_tr)        
        
        y_pred = model.predict(X_val)
        
        mse = mean_squared_error(y_val, y_pred)
        rmse.append( np.sqrt(mse) )
        
    return sum(rmse) / n_splits

In [76]:
def grid_search_CV(X, y, params_GridSearch):
    df_acc_by_params = pd.DataFrame(columns=list(params_GridSearch.keys()) + ['rmse'])
        
    for prms in tqdm( list(ParameterGrid(params_GridSearch)), ascii=True, desc='Params Tuning:', position=0, leave=True ):
                          
        rmse = random_forest_cross_val(X, y, prms, n_splits=5)
        
        data_dic = eval('{'+','.join([f"'{k}':[prms['{k}']]" for k in params_GridSearch.keys()])+'}')
        data_dic['rmse'] = [rmse]
        
        df_acc_by_params = pd.concat([df_acc_by_params,
                                      pd.DataFrame(data=data_dic)], 
                                      ignore_index=True)
        
    return df_acc_by_params

In [77]:
df_rmse_by_params = grid_search_CV(X, y, params_GridSearch)

Params Tuning::   0%|                                    | 0/90 [00:00<?, ?it/s]/Users/csr95/opt/anaconda3/envs/intelygenz-project/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Params Tuning::   1%|3                           | 1/90 [00:01<01:34,  1.07s/it]/Users/csr95/opt/anaconda3/envs/intelygenz-project/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Params Tuning::   2%|6                           | 2/90 [00:03<02:23,  1.63s/it]/Users/csr95/opt/anaconda3/envs/intelygenz-project/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
Params Tuning::   3%|9                           | 3/90 [00:07<04:16, 

In [78]:
df_rmse_by_params.dtypes

n_estimators         object
max_depth            object
min_samples_leaf     object
max_features         object
rmse                float64
dtype: object

In [80]:
df_rmse_by_params['n_estimators'] = df_rmse_by_params.n_estimators.astype(int)
df_rmse_by_params['max_depth'] = df_rmse_by_params.max_depth.astype(int)
df_rmse_by_params['min_samples_leaf'] = df_rmse_by_params.min_samples_leaf.astype(int)
df_rmse_by_params['max_features'] = df_rmse_by_params.max_features.astype(int)

In [81]:
df_rmse_by_params.head()

,n_estimators,max_depth,min_samples_leaf,max_features,rmse
0,100,2,1,1,294.576443
1,200,2,1,1,294.925897
2,500,2,1,1,294.958765
3,100,2,10,1,294.574529
4,200,2,10,1,294.925632


In [83]:
df_rmse_by_params.corr()['rmse']

n_estimators        0.003146
max_depth          -0.805514
min_samples_leaf    0.237544
max_features       -0.180496
rmse                1.000000
Name: rmse, dtype: float64

In [86]:
df_rmse_by_params.sort_values('rmse', ascending=True).head(10)

,n_estimators,max_depth,min_samples_leaf,max_features,rmse
72,100,20,1,1,270.508491
64,200,14,1,2,270.608165
83,500,20,1,2,270.621256
74,500,20,1,1,270.780135
65,500,14,1,2,270.904166
73,200,20,1,1,271.083283
56,500,14,1,1,271.144579
63,100,14,1,2,271.157648
81,100,20,1,2,271.229175
82,200,20,1,2,271.365252


In [91]:
best_params = df_rmse_by_params[ df_rmse_by_params.rmse==df_rmse_by_params.rmse.min() ]
best_params

,n_estimators,max_depth,min_samples_leaf,max_features,rmse
72,100,20,1,1,270.508491
